### 重新提取了原数据集的0.25   baseline :  0.715596

1、单独加入广告发布的不同用户数, 0.717474，效果有较大提升

2、加入兴趣特征， 0.716473，效果下降

3、加入年龄分布，0.717952，效果上升

4、加入教育分布,  0.715367，效果下降

5、加入消费能力分布，

4、删掉兴趣特征，0.717499，下降

5、加入广告点击率， 0.717974，效果上升

6、加入LBS历史点击率， 0.717536,效果下降

7、删掉广告点击率， 0.717347, 效果下降


6、分析广告投放和地理位置的关系

7、加入每个地理位置的人数特征, 效果下降

15、分析每个广告对应的地理位置分布


In [1]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

data = pd.read_csv('./datasets/train_data2.csv')
target = pd.read_csv('./datasets/train_target2.csv', names=['label'], header=None)

data.drop(data.columns[[0]], axis=1, inplace=True)  #删除某列函数
target = target.reset_index(drop=True)

one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility', 'education','gender','house','os','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']



for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
# split = StratifiedShuffleSplit(n_splits=1, test_size=0.12, random_state=42)
y = np.array(target).squeeze()
# for train_index, test_index in split.split(data,  y):
#     X_train = data.iloc[train_index]
#     X_test = data.iloc[test_index]
#     y_train = y[train_index]
#     y_test = y[test_index]
X_train,X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=2018)

In [3]:
data_clicked = X_train[y_train == 1]

In [4]:
# 处理联网类型特征
ct_train = X_train['ct'].values
ct_train = [m.split(' ') for m in ct_train]
ct_trains = []
for i in ct_train:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_trains.append(index)

ct_test = X_test['ct'].values
ct_test = [m.split(' ') for m in ct_test]
ct_tests = []
for i in ct_test:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_tests.append(index)


增加每个广告点击率特征

In [5]:
num_ad = X_train['aid'].value_counts().sort_index()
num_ad_clicked = data_clicked['aid'].value_counts().sort_index()

ratio = num_ad_clicked / num_ad

ratio_clicked = pd.DataFrame({
    'aid': ratio.index,
    'ratio_clicked' : ratio.values
})


In [6]:
X_train = pd.merge(X_train, ratio_clicked, on=['aid'], how='left')
X_test = pd.merge(X_test, ratio_clicked, on=['aid'], how='left')

增加每个广告出现次数

In [339]:
num_ad_post = X_train['aid'].value_counts().sort_index()

num_ad_post = pd.DataFrame({
    'aid': num_ad_post.index,
    'num_ad_post' : num_ad_post.values
})

In [340]:
X_train = pd.merge(X_train, num_ad_post, on=['aid'], how='left')
X_test = pd.merge(X_test, num_ad_post, on=['aid'], how='left')

增加每个广告推送给不同的用户数

In [7]:
num_advertise_touser = X_train.groupby('aid').uid.nunique()
num_advertise_touser = pd.DataFrame({
    'aid': num_advertise_touser.index,
    'num_advertise_touser' : num_advertise_touser.values
})
X_train = pd.merge(X_train, num_advertise_touser, on=['aid'], how='left')
X_test = pd.merge(X_test, num_advertise_touser, on=['aid'], how='left')

加入推广计划转化率

In [55]:
num_campaign = X_train['campaignId'].value_counts().sort_index()
num_campaign_clicked = data_clicked['campaignId'].value_counts().sort_index()

ratio_num_campaign = num_campaign_clicked / num_campaign

ratio_num_campaign = pd.DataFrame({
    'campaignId': ratio_num_campaign.index,
    'ratio_num_campaign' : ratio_num_campaign.values
})
X_train = pd.merge(X_train, ratio_num_campaign, on=['campaignId'], how='left')
X_test = pd.merge(X_test, ratio_num_campaign, on=['campaignId'], how='left')

加入用户兴趣总数

In [56]:
train_all_interest = X_train[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
test_all_interest = X_test[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
num_all_interest_train = []
num_all_interest_test = []
aids = []
train_array = np.array(train_all_interest)
test_array = np.array(test_all_interest)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_train.append(num)
    
    
for i in range(test_array.shape[0]):
    num = 0
    inter = test_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_test.append(num)

In [57]:
num_all_interest_train = pd.DataFrame(num_all_interest_train, columns=['num_all_interest'])
num_all_interest_test = pd.DataFrame(num_all_interest_test, columns=['num_all_interest'])

X_train = pd.concat([X_train, num_all_interest_train], axis=1)
X_test = pd.concat([X_test, num_all_interest_test], axis=1)

分析每个广告对应的人群年龄分布, 教育水平分布，消费能力分布

In [8]:
def get_ad_toother(typename):
    num_ad_totype = X_train.groupby('aid')[typename].value_counts()
    num_ad_totype_clicked = data_clicked.groupby('aid')[typename].value_counts()
    ratio_num_ad_totype = num_ad_totype_clicked / num_ad_totype
    list_num_ad_totype = []
    num_adid = X_train['aid'].value_counts().sort_index().index
    for aid_out in num_adid:
        dict_buf = {}
        dict_num_ad_totype = {}
        dict_num_ad_totype['aid'] = aid_out
        for i, j in ratio_num_ad_totype.items():
            aid = i[0]
            feature = i[1]
            if(aid == aid_out):
                dict_buf[feature] = float("%.5f" % j)
        fea_name = 'num_ad_to'+typename
        dict_num_ad_totype[fea_name] = dict_buf
        list_num_ad_totype.append(dict_num_ad_totype)
    return list_num_ad_totype

In [9]:
list_num_ad_toedu = get_ad_toother('education')
list_num_ad_toedu = pd.DataFrame(list_num_ad_toedu)
X_train = pd.merge(X_train, list_num_ad_toedu, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_toedu, on=['aid'], how='left')
X_train['ratio_num_ad_toedu'] = [ j.get(i, 0) for i, j in X_train[['education', 'num_ad_toeducation']].values]
X_test['ratio_num_ad_toedu'] = [ j.get(i, 0) for i, j in X_test[['education', 'num_ad_toeducation']].values]

In [ ]:
list_num_ad_toage = get_ad_toother('age')
list_num_ad_toage = pd.DataFrame(list_num_ad_toage)
X_train = pd.merge(X_train, list_num_ad_toage, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_toage, on=['aid'], how='left')
X_train['ratio_num_ad_toage'] = [ j.get(i, 0) for i, j in X_train[['age', 'num_ad_toage']].values]
X_test['ratio_num_ad_toage'] = [ j.get(i, 0) for i, j in X_test[['age', 'num_ad_toage']].values]

In [ ]:
list_num_ad_toconsume = get_ad_toother('consumptionAbility')
list_num_ad_toconsume = pd.DataFrame(list_num_ad_toconsume)
X_train = pd.merge(X_train, list_num_ad_toconsume, on=['aid'], how='left')
X_test = pd.merge(X_test, list_num_ad_toconsume, on=['aid'], how='left')
X_train['ratio_num_ad_toconsume'] = [ j.get(i, 0) for i, j in X_train[['consumptionAbility', 'num_ad_toconsumptionAbility']].values]
X_test['ratio_num_ad_toconsume'] = [ j.get(i, 0) for i, j in X_test[['consumptionAbility', 'num_ad_toconsumptionAbility']].values]

In [ ]:
X_train.head()

加入地理位置所对应的用户人数

In [58]:
num_lbs_user = X_train.groupby('LBS').uid.nunique()
num_lbs_user = pd.DataFrame({
    'LBS': num_lbs_user.index,
    'num_lbs_user' : num_lbs_user.values
})
X_train = pd.merge(X_train, num_lbs_user, on=['LBS'], how='left')
X_test = pd.merge(X_test, num_lbs_user, on=['LBS'], how='left')

加入用户所在LBS的历史点击率

In [14]:
num_lbs = X_train.groupby('LBS').uid.nunique()
num_lbs_clicked = data_clicked.groupby('LBS').uid.nunique()
ratio_num_lbs = num_lbs_clicked / num_lbs
ratio_num_lbs = ratio_num_lbs.fillna(0)
ratio_num_lbs = pd.DataFrame({
    'LBS': ratio_num_lbs.index,
    'ration_num_LBS' : ratio_num_lbs.values
})
X_train = pd.merge(X_train, ratio_num_lbs, on=['LBS'], how='left')
X_test = pd.merge(X_test, ratio_num_lbs, on=['LBS'], how='left')

加入用户的历史接收的广告次数与历史点击次数，以及历史点击率

In [56]:
num_user_getad = X_train.groupby('uid').aid.nunique().sort_index()
num_user_clicked = data_clicked.groupby('uid').aid.nunique().sort_index()

num_user_getad = pd.DataFrame({
    'uid': num_user_getad.index,
    'num_user_getad' : num_user_getad.values
})

num_user_clicked = pd.DataFrame({
    'uid': num_user_clicked.index,
    'num_user_clicked' : num_user_clicked.values
})

In [182]:
X_train = pd.merge(X_train, num_user_clicked, on=['uid'], how='left')
X_test = pd.merge(X_test, num_user_clicked, on=['uid'], how='left')

In [ ]:
X_train.fillna(0, inplace=True)

In [ ]:
X_test.fillna(0, inplace=True)

In [41]:


# X_train_encoded= np.hstack((X_train_encoded, ct_trains))
# X_test_encoded= np.hstack((X_test_encoded, ct_tests))

# , 'num_common_interest2', 'num_common_interest1', 'num_common_interest5' 'num_advertise_touser' 'ration_num_LBS' 
X_train_encoded = ct_trains
X_test_encoded = ct_tests

In [42]:
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(data[feature].reshape(-1, 1))
    train_a=oc_encoder.transform(X_train[feature].values.reshape(-1, 1))
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a=oc_encoder.transform(X_test[feature].values.reshape(-1, 1))
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [162]:
X_train_encoded.toarray().shape

(703905, 1183)

In [43]:
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']
ct_encoder = CountVectorizer(min_df=0.0009)
for feature in vector_feature:
    ct_encoder.fit(data[feature])
    train_a = ct_encoder.transform(X_train[feature])
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a = ct_encoder.transform(X_test[feature])
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

In [100]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[['num_advertise_touser','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 'num_common_topic1',
                      'ratio_num_ad_toage', 'ratio_num_ad_toconsume']].values)
train_encoded = scaler.transform(X_train[['num_advertise_touser','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 
                      'num_common_topic1', 'ratio_num_ad_toage', 'ratio_num_ad_toconsume']].values)

test_encoded = scaler.transform(X_test[['num_advertise_touser','num_common_interest2', 'num_common_interest1', 'num_common_interest5', 
                        'num_common_topic1', 'ratio_num_ad_toage', 'ratio_num_ad_toconsume']].values)

# scaler.fit(X_train[['num_advertise_touser']].values)
# train_encoded = scaler.transform(X_train[['num_advertise_touser']].values)
# test_encoded = scaler.transform(X_test[['num_advertise_touser']].values)  'ratio_num_ad_tocedu'
# ,'num_common_interest2', 'num_common_interest1', 'num_common_interest5', 'num_common_topic1'  'ratio_num_ad_toconsume' 
X_train_encoded2= sparse.hstack((X_train_encoded, train_encoded))
X_test_encoded2= sparse.hstack((X_test_encoded, test_encoded))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [101]:
# y_train = np.array(target).squeeze()
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=0.9,
        max_depth=-1, n_estimators=300, objective='binary',
        subsample=0.7, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.05, min_child_weight=60, random_state=2018, n_jobs=-1
    )
clf.fit(X_train_encoded2, y_train, eval_set=[(X_test_encoded2, y_test)], eval_metric='auc',early_stopping_rounds=50)

KeyboardInterrupt: 

In [ ]:
def get_common_interest(type_name, ratio):
    num_adid = data_clicked['aid'].value_counts().sort_index().index
    num_aid_clicked = dict(data_clicked['aid'].value_counts().sort_index())
    num_user_clicksameAd_interest = data_clicked.groupby('aid')[type_name].value_counts()
    dict_interest = {}
    for adid in num_adid:
        dict_buf = {}
        for interest in num_user_clicksameAd_interest.items():
            index = interest[0]
            if index[0] == adid:
                number = interest[1]
                detail = index[1]
                detail = detail.split(' ')
                for det in detail:
                    if det not in dict_buf:
                        dict_buf[det] = number
                    else:
                        dict_buf[det] += number
        dict_interest[adid] = dict_buf
    dict_common_interest = []
    for adid, dict_inter in dict_interest.items():
        dict_common_buf = {}
        dict_common_buf['aid'] = adid
        common_inter = []
        ad_total = num_aid_clicked[adid] - dict_inter.get('-1', 0)
        if '-1' in dict_inter:
            dict_inter.pop('-1')
        for id_inter, num in dict_inter.items():
            if num >= ad_total*ratio:
                common_inter.append(id_inter)
        str_name = 'common_'+type_name
        dict_common_buf[str_name] = common_inter
        dict_common_interest.append(dict_common_buf)
    return dict_common_interest

获取相同的兴趣ID2

In [ ]:
dict_common_interest2 = get_common_interest('interest2', 0.25)
df_common_interest2 = pd.DataFrame(dict_common_interest2)
X_train = pd.merge(X_train, df_common_interest2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest2, on=['aid'], how='left')

获取相同的兴趣ID1

In [ ]:
dict_common_interest1 = get_common_interest('interest1', 0.25)
df_common_interest1 = pd.DataFrame(dict_common_interest1)
X_train = pd.merge(X_train, df_common_interest1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest1, on=['aid'], how='left')

获取相同的兴趣ID5

In [ ]:
dict_common_interest5 = get_common_interest('interest5', 0.25)
df_common_interest5 = pd.DataFrame(dict_common_interest5)
X_train = pd.merge(X_train, df_common_interest5, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_interest5, on=['aid'], how='left')

In [ ]:
X_train['num_common_interest2'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest2', 'common_interest2']].values]
X_test['num_common_interest2'] = [len(set(i.split(' ')).intersection(set(j))) /len(j) for i, j in X_test[['interest2', 'common_interest2']].values]

In [ ]:
X_train['num_common_interest1'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest1', 'common_interest1']].values]
X_test['num_common_interest1'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_test[['interest1', 'common_interest1']].values]

In [ ]:
X_train['num_common_interest5'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_train[['interest5', 'common_interest5']].values]
X_test['num_common_interest5'] = [len(set(i.split(' ')).intersection(set(j))) / len(j) for i, j in X_test[['interest5', 'common_interest5']].values]

In [79]:
X_train['num_common_all'] = X_train[['num_common_interest5', 'num_common_interest2', 'num_common_interest1']].sum(axis=1)
X_test['num_common_all'] = X_test[['num_common_interest5', 'num_common_interest2', 'num_common_interest1']].sum(axis=1)

提取关键字1,2

In [122]:
dict_common_kw1 = get_common_interest('kw1', 0.05)
df_common_kw1 = pd.DataFrame(dict_common_kw1)

In [123]:
X_train = pd.merge(X_train, df_common_kw1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_kw1, on=['aid'], how='left')

In [124]:
X_train['num_common_kw1'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['kw1', 'common_kw1']].values ]
X_test['num_common_kw1'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['kw1', 'common_kw1']].values ]

In [125]:
dict_common_kw2 = get_common_interest('kw2', 0.05)
df_common_kw2 = pd.DataFrame(dict_common_kw2)
X_train = pd.merge(X_train, df_common_kw2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_kw2, on=['aid'], how='left')
X_train['num_common_kw2'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['kw2', 'common_kw2']].values ]
X_test['num_common_kw2'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['kw2', 'common_kw2']].values ]

提取主题1,2

In [ ]:
dict_common_topic1 = get_common_interest('topic1', 0.1)
df_common_topic1 = pd.DataFrame(dict_common_topic1)
X_train = pd.merge(X_train, df_common_topic1, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_topic1, on=['aid'], how='left')
X_train['num_common_topic1'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['topic1', 'common_topic1']].values ]
X_test['num_common_topic1'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['topic1', 'common_topic1']].values ]

In [127]:
dict_common_topic2 = get_common_interest('topic2', 0.05)
df_common_topic2 = pd.DataFrame(dict_common_topic2)
X_train = pd.merge(X_train, df_common_topic2, on=['aid'], how='left')
X_test = pd.merge(X_test, df_common_topic2, on=['aid'], how='left')
X_train['num_common_topic2'] = [len(set(i.split(' ')).intersection(set(j))) / (len(j)+1) for i, j in X_train[['topic2', 'common_topic2']].values ]
X_test['num_common_topic2'] = [len(set(i.split(' ')).intersection(set(j))) /  (len(j)+1) for i, j in X_test[['topic2', 'common_topic2']].values ]

In [86]:
X_train.head()

,aid,uid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType,LBS,...,num_ad_toeducation,ratio_num_ad_toedu,num_ad_toage,ratio_num_ad_toage,common_interest2,common_interest1,common_interest5,num_common_interest2,num_common_interest1,num_common_interest5
0,411,25980816,36,83,21,8,6,0,0,573,...,"{0: 0.09407, 1: 0.05341, 2: 0.05353, 3: 0.0502...",0.05029,"{0: 0.18, 1: 0.03039, 2: 0.06225, 3: 0.04074, ...",0.06225,"[30, 73, 70, 46]","[36, 11, 70, 76, 109, 75, 29, 46, 49]","[46, 6, 59, 131, 8, 76, 93, 116, 132, 121, 129...",0.0,0.333333,0.000000
1,1918,7431028,78,128,162,6,1,0,0,676,...,"{0: 0.10204, 1: 0.05081, 2: 0.08037, 3: 0.0639...",0.04388,"{1: 0.09115, 2: 0.17875, 3: 0.04918, 4: 0.0378...",0.04945,"[24, 30, 52, 21, 73, 74, 70]","[36, 11, 75, 29, 70, 76, 93, 46, 109, 28, 77, ...","[100, 46, 37, 6, 59, 80, 129, 78, 30, 131, 8, ...",0.0,0.550000,0.375000
2,1468,4312799,12,3,152,6,16,0,0,367,...,"{0: 0.03614, 1: 0.05738, 2: 0.07126, 3: 0.0717...",0.07126,"{0: 0.11111, 1: 0.05028, 2: 0.10325, 3: 0.0239...",0.01995,"[24, 30, 73, 9, 70, 10, 46, 4]","[36, 11, 70, 76, 75, 29, 109, 119, 47, 9, 93, ...","[100, 46, 37, 131, 8, 6, 59, 78, 30, 109, 50, ...",0.0,0.210526,0.571429
3,960,69244796,9,48,159,5,4,14,3,156,...,"{0: 0.02913, 1: 0.04698, 2: 0.05079, 3: 0.0595...",0.05841,"{0: nan, 1: 0.05945, 2: 0.05981, 3: 0.05433, 4...",0.04044,[30],"[36, 11, 70, 76, 75, 29, 93, 46, 109, 59, 49, ...","[100, 131, 37, 116, 8, 50, 6, 42, 46, 121, 59,...",0.0,0.823529,0.233333
4,660,1747755,76,123,106,12,6,0,0,594,...,"{0: 0.09091, 1: 0.06395, 2: 0.04815, 3: 0.0336...",0.05698,"{1: 0.06029, 2: 0.07923, 3: 0.03609, 4: 0.0431...",0.05652,"[73, 24, 30, 9, 21, 6, 46, 70, 4]","[75, 29, 36, 11, 109, 49, 6, 114, 116, 59, 118...","[100, 109, 6, 46, 59, 116, 13, 71, 132, 121, 1...",0.0,0.714286,0.687500


In [66]:
# X_train.drop(['common_topic1_x','common_topic1_y'] ,axis=1, inplace=True)
# X_test.drop(['common_topic1_x','common_topic1_y'] ,axis=1, inplace=True)

# X_test.drop(['common_interest2_x','common_interest1_x','interest2_new','interest1_new','common_interest2_y'
#              ,'common_interest1_y','common_interest2','common_interest1'], axis=1, inplace=True)

# X_test.drop(['num_common_interest','common_interest1_x','num_common_interest1','common_interest1_y','common_interest1','common_interest2_y'],
#             axis=1, inplace=True)
# X_train.drop(['num_common_interest','common_interest1_x','num_common_interest1','common_interest1_y','common_interest1','common_interest2_y'],
#             axis=1, inplace=True)

In [206]:
for i, j in X_train[['interest2', 'common_interest2']].values:
    print(i, j)
    break

22 1 70 ['30', '73', '70']


In [83]:
train_interest1_new = []
for i in train_user_interest1.values:
    inter2_buf = []
    int2 = i[0].split(' ')
    common = i[1]
    for det in int2:
        if det in common:
            inter2_buf.append(det)
    inter2_buf.extend(int2)
    train_interest1_new.append(inter2_buf)

In [84]:
test_interest1_new = []
for i in test_user_interest1.values:
    inter2_buf = []
    int2 = i[0].split(' ')
    common = i[1]
    for det in int2:
        if det in common:
            inter2_buf.append(det)
    inter2_buf.extend(int2)
    test_interest1_new.append(inter2_buf)

In [85]:
train_interest2_new = [' '.join(m) for m in train_interest2_new]
X_train['interest2_new'] = train_interest2_new

In [86]:
test_interest2_new = [' '.join(m) for m in test_interest2_new]
X_test['interest2_new'] = test_interest2_new

In [87]:
train_interest1_new = [' '.join(m) for m in train_interest1_new]
X_train['interest1_new'] = train_interest2_new

In [88]:
test_interest1_new = [' '.join(m) for m in test_interest1_new]
X_test['interest1_new'] = test_interest1_new

In [89]:
concat_new = pd.concat([X_train[['uid', 'interest2_new']], X_test[['uid', 'interest2_new']]])

In [91]:
concat_new1 = pd.concat([X_train[['uid', 'interest1_new']], X_test[['uid', 'interest1_new']]])